In [20]:
!pip3 install bs4

home_page = "https://stackexchange.com/?pagesize=50"
import requests
import json
from bs4 import BeautifulSoup
import logging
import datetime
import dateparser

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:
with open("sent.json","w") as file:
    json.dump([("https://www.imdb.com/title/tt0390384","Primer","16 January 2004")], file) 

In [151]:
!ls

 draft.ipynb  'se parsing.ipynb'   se-server.py
 nohup.out     se-parser.py	   sent.json


In [154]:
import se_parser

print(se_parser.workflow())

/home/hwlcrd/mounted/pets/bot factory/se-best/se_parser.py:70: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 70 of the file /home/hwlcrd/mounted/pets/bot factory/se-best/se_parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(page)


[['https://www.imdb.com/title/tt0390384', 'Primer', '16 January 2004'], ['https://academia.stackexchange.com/questions/149759/is-relocating-necessary-in-academia', 'Is Relocating Necessary in Academia?', '2020-05-29 02:56:01Z'], ['https://codegolf.stackexchange.com/questions/205422/zero-the-byte-eventually', 'Zero the byte (eventually)', '2020-05-30 00:48:05Z'], ('https://money.stackexchange.com/questions/125998/how-can-a-client-pay-me-for-freelance-work-if-i-dont-want-him-to-know-my-name-a', "How can a client pay me for freelance work if I don't want him to know my name and he is uncomfortable using bitcoin?", '2020-05-28 20:49:09Z')]
Q: How can a client pay me for freelance work if I don't want him to know my name and he is uncomfortable using bitcoin?


I've been contacted on a specialized Internet forum and offered a well-paid one-off job to translate a highly specialized text, and I'm happy to accept the offer, but there's a problem: I have always posted on that forum under an ano

/home/hwlcrd/mounted/pets/bot factory/se-best/se_parser.py:47: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 47 of the file /home/hwlcrd/mounted/pets/bot factory/se-best/se_parser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(page)


In [146]:
def workflow():
    track = tracker()
    track.load()
    posts = post_getter()
    for post in posts:
        # check post for being previously sent
        if track.check_post(post):
            # download post
            payload = gather_post(post)
            track.save_post(payload)
            track.save()
            print(f"{track.data}")
            return generate_text(payload)

In [143]:
def generate_text(payload):
    # output {url, title, date, question, username}
    return f"""Q: {payload['title']}

{payload['question']}

{payload['username']}, {payload['date']}
"""

In [144]:
def post_getter():
    # Download stack exchange hot page
    home_page = "https://stackexchange.com/?pagesize=50"
    page = requests.get(home_page).text
    soup = BeautifulSoup(page)
    # Parse page
    new = soup.find_all("a",class_='question-link')
    # Extract posts and return data
    posts = [elem["href"] for elem in new]
    return posts

In [145]:
def gather_post(url):
    # download post
    # output {url, title, date, question, username}
    page = requests.get(url).text
    soup = BeautifulSoup(page)
    title = soup.find("a",class_='question-hyperlink').get_text()
    #answer = soup.find("div",class_='question').get_text().split("share")[0]
    question = soup.find("div",class_='postcell post-layout--right').find("div","post-text").get_text() #.findAll("div")#.get_text()
    username = soup.find("div",class_='user-details', itemprop="author").get_text().strip("\n").split("\n")[0]
    date = soup.find("span", class_='relativetime')['title']
    return {
        "title":title,
        "question":clear_text(question),
        "url":url,
        "username":username,
        "date":date
    }

def clear_text(text):
    if len(text) > 400:
        return text[:400] + "..."
    return text
    
    

In [142]:
class tracker:
    def __init__(self):
        self.data_file = "sent.json"
        self.data = []
        self.week = datetime.timedelta(days=7)
    
    def load(self):
        # load data if not loaded
        # data format list of (url, title, date)
        if not self.data:
            with open(self.data_file,"r") as file:
                self.data = json.load(file)
                
    def save(self):
        # load data if not loaded
        # data format list of (url, title, date)
        with open(self.data_file,"w") as file:
            json.dump(self.data, file)
    
    def check_post(self, url):
        for post in self.data:
            if post[0]==url:
                return False
        return True
    
    def ttl(self):
        now = datetime.datetime.now()
        self.data = list(filter(lambda x: now-dateparser.parse(x[2]) < self.week, self.data))
        
    
    def save_post(self, post):
        self.data.append((post['url'],post['title'],post['date']))

In [108]:
# testing tracker


In [109]:
track.data


[['https://www.imdb.com/title/tt0390384', 'Primer', '16 January 2004']]

In [111]:
track = tracker()
print(f"1: {track.data}")
track.load()
print(f"2: {track.data}")
print(f"3: {track.check_post('https://www.imdb.com/title/tt0390384')}")
print(f"4: {track.check_post('https://www.google.com')}")
track.ttl()
print(f"5 ttl: {track.data}")
track.save()

1: 
2: [['https://www.imdb.com/title/tt0390384', 'Primer', '16 January 2004']]
3: False
4: True
5 ttl: []


In [27]:
datetime.timedelta(days=7)

datetime.timedelta(days=7)

In [30]:
datetime.datetime.now()-dateparser.parse("1 January 2004")

datetime.timedelta(days=5994, seconds=44645, microseconds=268353)

In [ ]:
list(filter(lambda x: x%2, [1,2,3]))

In [ ]:
# (url, title, date)

In [14]:
import json

In [15]:
json.dumps([(1,2,3)])

'[[1, 2, 3]]'

In [10]:
def post_getter():
    # Download stack exchange hot page
    home_page = "https://stackexchange.com/?pagesize=50"
    page = requests.get(home_page).text
    soup = BeautifulSoup(page)
    # Parse page
    new = soup.find_all("a",class_='question-link')
    # Extract posts and return data
    posts = [{"url":elem["href"],
             "title":elem.text.strip(" \r\n")} for elem in new]
    return posts

In [ ]:
def decider():
    

In [33]:
post_getter()

['https://codegolf.stackexchange.com/questions/205422/zero-the-byte-eventually',
 'https://academia.stackexchange.com/questions/149759/is-relocating-necessary-in-academia',
 'https://money.stackexchange.com/questions/125998/how-can-a-client-pay-me-for-freelance-work-if-i-dont-want-him-to-know-my-name-a',
 'https://worldbuilding.stackexchange.com/questions/177358/how-would-tiny-dragons-strike-terror',
 'https://retrocomputing.stackexchange.com/questions/15004/what-languages-are-better-fit-for-generating-efficient-code-for-8-bit-cpus-than',
 'https://workplace.stackexchange.com/questions/158641/can-i-get-in-trouble-for-smoking-while-i-was-on-a-video-call-while-i-was-working',
 'https://codegolf.stackexchange.com/questions/205425/compute-xkcds-wrong-times-table',
 'https://math.stackexchange.com/questions/3697350/how-long-will-it-take-to-catch-up-to-a-stream',
 'https://english.stackexchange.com/questions/535948/what-is-the-p-in-nope-called',
 'https://worldbuilding.stackexchange.com/ques

In [7]:
post_getter()[0]

# url -> href
# title -> text

<a class="question-link" href="https://codegolf.stackexchange.com/questions/205422/zero-the-byte-eventually" title="This question has been arbitrarily awarded 55.419 hotness points">
                    Zero the byte (eventually)
                </a>

In [9]:
post_getter()[0].text.strip(" \r\n")


'Zero the byte (eventually)'